<a href="https://colab.research.google.com/github/rze7ud/DS3001_Final_Project/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install python-dotenv
!pip install cfbd

Use a .env file with cfbd API_KEY (https://collegefootballdata.com/key) \
\
.env:\
API_KEY=" "

In [55]:
import cfbd
from cfbd.rest import ApiException
from __future__ import print_function
import json

import pandas as pd

from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv('API_KEY')

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = API_KEY
configuration.api_key_prefix['Authorization'] = 'Bearer'

# 2. Get Total Season Stats / Weekly Season Stats

In [56]:
def get_stats(season, team):
    api_instance = cfbd.StatsApi(cfbd.ApiClient(configuration))
    try:
        api_response = api_instance.get_team_season_stats(year=season, team=team)
        try:
            return api_response
        except IndexError:
            return None
    except ApiException as e:
        print("Exception when calling StatsApi->get_: %s\n" % e)

In [57]:
def get_cumulative_stats(season, team, week):
    api_instance = cfbd.StatsApi(cfbd.ApiClient(configuration))
    try:

        if week != 1:
            api_response = api_instance.get_team_season_stats(year=season, team=team, end_week=week-1)
        else:
            return None
        try:
            return api_response
        except IndexError:
            return None
    except ApiException as e:
        print("Exception when calling StatsApi->get_: %s\n" % e)

In [58]:
def get_data(team, start_year, end_year, isCumulative):
    api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
    years = list(range(start_year, end_year+1))
    df = pd.DataFrame()
    for year in years:
        try:
            api_response = api_instance.get_games(year=year, team=team)
            for game in api_response:
                season = game.season
                week = game.week
                if game.home_team == f"{team}":
                    home = 'True'
                    team_points = game.home_points
                    opp = game.away_team
                    opp_points = game.away_points
                else:
                    home = 'False'
                    team_points = game.away_points
                    opp = game.home_team
                    opp_points = game.home_points
                if str(team_points) != 'None' and str(opp_points) != 'None':
                    if int(str(team_points)) > int(str(opp_points)):
                        win = 'True'
                    else:
                        win = 'False'
                else:
                    win = None
                api_response = get_stats(season, opp)
                if isCumulative:
                    api_team_response = get_cumulative_stats(season, team, week)
                else:
                    api_team_response = get_stats(season, team)
                if api_response and api_team_response is not None:
                    data_dict = {}
                    data_dict['season'] = season
                    data_dict['week'] = week
                    data_dict['home'] = home
                    data_dict['win'] = win
                    data_dict[f'{team}_points'] = team_points
                    data_dict['opp'] = opp
                    data_dict[f'opp_points'] = opp_points
                    for i in range(len(api_response)):
                        data_dict[f"opp_{api_response[i].stat_name}"] = api_response[i].stat_value
                    for i in range(len(api_team_response)):
                        data_dict[api_team_response[i].stat_name] = api_team_response[i].stat_value
                    new_row = pd.DataFrame([data_dict])
                    df = pd.concat([df, new_row], ignore_index=True)
        except ApiException as e:
            print("Exception when calling GamesApi->get_games: %s\n" % e)
    return df

In [59]:
clemson_2001_2024 = get_data('Clemson', 2001, 2024, False)
clemson_2001_2024.to_csv('clemson_2001_2024.csv', index=False)

clemson_weekly_2001_2024 = get_data('Clemson', 2001, 2024, True)
clemson_weekly_2001_2024.to_csv('clemson_weekly_2001_2024.csv', index=False)

In [60]:
smu_2001_2024 = get_data('SMU', 2001, 2024, False)
smu_2001_2024.to_csv('smu_2001_2024.csv', index=False)

smu_weekly_2001_2024 = get_data('SMU', 2001, 2024, True)
smu_weekly_2001_2024.to_csv('smu_weekly_2001_2024.csv', index=False)

# 3a. Get Advanced Season Stats

In [ ]:
def get_advanced_stats(season, team):
    api_instance = cfbd.StatsApi(cfbd.ApiClient(configuration))
    try:
        api_response = api_instance.get_advanced_team_season_stats(year=season, team=team, exclude_garbage_time=True)
        try:
            return api_response[0]
        except IndexError:
            return None
    except ApiException as e:
        print("Exception when calling StatsApi->get_: %s\n" % e)

In [ ]:
def get_advanced_opp_data(team, start_year, end_year):
    api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
    years = list(range(start_year, end_year+1))

    columns = ['season', 'week', 'home', 'win', f'{team.lower()}_points', 'opp', 'opp_points', 'opp_conference',

               'opp_def_explosiveness', 'opp_def_fp_avg_pred_points', 'opp_def_fp_avg_start',
               'opp_def_havoc_db', 'opp_def_havoc_7', 'opp_def_havoc_total',
               'opp_def_line_yards', 'opp_def_open_field_yards',
               'opp_def_pp_explosiveness', 'opp_def_pp_ppa', 'opp_def_pp_rate', 'opp_def_pp_success_rate',
               'opp_def_points_per_opportunity', 'opp_def_power_success', 'opp_def_ppa',
               'opp_def_rp_explosiveness', 'opp_def_rp_ppa', 'opp_def_rp_rate', 'opp_def_rp_success_rate',
               'opp_def_second_level_yards', 'opp_def_stuff_rate', 'opp_def_success_rate', 'opp_def_total_ppa',

               'opp_off_explosiveness', 'opp_off_fp_avg_pred_points', 'opp_off_fp_avg_start',
               'opp_off_havoc_db', 'opp_off_havoc_7', 'opp_off_havoc_total',
               'opp_off_line_yards', 'opp_off_open_field_yards',
               'opp_off_pp_explosiveness', 'opp_off_pp_ppa', 'opp_off_pp_rate', 'opp_off_pp_success_rate',
               'opp_off_points_per_opportunity', 'opp_off_power_success', 'opp_off_ppa',
               'opp_off_rp_explosiveness', 'opp_off_rp_ppa', 'opp_off_rp_rate', 'opp_off_rp_success_rate',
               'opp_off_second_level_yards', 'opp_off_stuff_rate', 'opp_off_success_rate', 'opp_off_total_ppa']
    df = pd.DataFrame(columns=columns)
    for year in years:
        try:
            api_response = api_instance.get_games(year=year, team=team)
            for game in api_response:
                season = game.season
                week = game.week
                if game.home_team == f"{team}":
                    home = 'True'
                    team_points = game.home_points
                    opp = game.away_team
                    opp_points = game.away_points
                else:
                    home = 'False'
                    team_points = game.away_points
                    opp = game.home_team
                    opp_points = game.home_points
                if str(team_points) != 'None' and str(opp_points) != 'None':
                    if int(str(team_points)) > int(str(opp_points)):
                        win = 'True'
                    else:
                        win = 'False'
                else:
                    win = None
                api_response = get_advanced_stats(season, opp)
                if api_response is not None:
                    opp_conference = api_response.conference

                    opp_def_explosiveness = api_response.defense.explosiveness
                    opp_def_fp_avg_pred_points = api_response.defense.field_position.average_predicted_points
                    opp_def_fp_avg_start = api_response.defense.field_position.average_start
                    opp_def_havoc_db = api_response.defense.havoc.db
                    opp_def_havoc_7 = api_response.defense.havoc.front_seven
                    opp_def_havoc_total = api_response.defense.havoc.total
                    opp_def_line_yards = api_response.defense.line_yards
                    opp_def_open_field_yards = api_response.defense.open_field_yards
                    opp_def_pp_explosiveness = api_response.defense.passing_plays.explosiveness
                    opp_def_pp_ppa = api_response.defense.passing_plays.ppa
                    opp_def_pp_rate = api_response.defense.passing_plays.rate
                    opp_def_pp_success_rate = api_response.defense.passing_plays.success_rate
                    opp_def_points_per_opportunity = api_response.defense.points_per_opportunity
                    opp_def_power_success = api_response.defense.power_success
                    opp_def_ppa = api_response.defense.ppa
                    opp_def_rp_explosiveness = api_response.defense.rushing_plays.explosiveness
                    opp_def_rp_ppa = api_response.defense.rushing_plays.ppa
                    opp_def_rp_rate = api_response.defense.rushing_plays.rate
                    opp_def_rp_success_rate = api_response.defense.rushing_plays.success_rate
                    opp_def_second_level_yards = api_response.defense.second_level_yards
                    opp_def_stuff_rate = api_response.defense.stuff_rate
                    opp_def_success_rate = api_response.defense.success_rate
                    opp_def_total_ppa = api_response.defense.total_ppa

                    opp_off_explosiveness = api_response.offense.explosiveness
                    opp_off_fp_avg_pred_points = api_response.offense.field_position.average_predicted_points
                    opp_off_fp_avg_start = api_response.offense.field_position.average_start
                    opp_off_havoc_db = api_response.offense.havoc.db
                    opp_off_havoc_7 = api_response.offense.havoc.front_seven
                    opp_off_havoc_total = api_response.offense.havoc.total
                    opp_off_line_yards = api_response.offense.line_yards
                    opp_off_open_field_yards = api_response.offense.open_field_yards
                    opp_off_pp_explosiveness = api_response.offense.passing_plays.explosiveness
                    opp_off_pp_ppa = api_response.offense.passing_plays.ppa
                    opp_off_pp_rate = api_response.offense.passing_plays.rate
                    opp_off_pp_success_rate = api_response.offense.passing_plays.success_rate
                    opp_off_points_per_opportunity = api_response.offense.points_per_opportunity
                    opp_off_power_success = api_response.offense.power_success
                    opp_off_ppa = api_response.offense.ppa
                    opp_off_rp_explosiveness = api_response.offense.rushing_plays.explosiveness
                    opp_off_rp_ppa = api_response.offense.rushing_plays.ppa
                    opp_off_rp_rate = api_response.offense.rushing_plays.rate
                    opp_off_rp_success_rate = api_response.offense.rushing_plays.success_rate
                    opp_off_second_level_yards = api_response.offense.second_level_yards
                    opp_off_stuff_rate = api_response.offense.stuff_rate
                    opp_off_success_rate = api_response.offense.success_rate
                    opp_off_total_ppa = api_response.offense.total_ppa

                    df.loc[len(df)] = [season, week, home, win, team_points, opp, opp_points, opp_conference,

                                        opp_def_explosiveness, opp_def_fp_avg_pred_points, opp_def_fp_avg_start,
                                        opp_def_havoc_db, opp_def_havoc_7, opp_def_havoc_total,
                                        opp_def_line_yards, opp_def_open_field_yards,
                                        opp_def_pp_explosiveness, opp_def_pp_ppa, opp_def_pp_rate, opp_def_pp_success_rate,
                                        opp_def_points_per_opportunity, opp_def_power_success, opp_def_ppa,
                                        opp_def_rp_explosiveness, opp_def_rp_ppa, opp_def_rp_rate, opp_def_rp_success_rate,
                                        opp_def_second_level_yards, opp_def_stuff_rate, opp_def_success_rate, opp_def_total_ppa,

                                        opp_off_explosiveness, opp_off_fp_avg_pred_points, opp_off_fp_avg_start,
                                        opp_off_havoc_db, opp_off_havoc_7, opp_off_havoc_total,
                                        opp_off_line_yards, opp_off_open_field_yards,
                                        opp_off_pp_explosiveness, opp_off_pp_ppa, opp_off_pp_rate, opp_off_pp_success_rate,
                                        opp_off_points_per_opportunity, opp_off_power_success, opp_off_ppa,
                                        opp_off_rp_explosiveness, opp_off_rp_ppa, opp_off_rp_rate, opp_off_rp_success_rate,
                                        opp_off_second_level_yards, opp_off_stuff_rate, opp_off_success_rate, opp_off_total_ppa]
        except ApiException as e:
            print("Exception when calling GamesApi->get_games: %s\n" % e)
    return df

In [ ]:
def get_advanced_data(team, start_year, end_year):
    api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
    years = list(range(start_year, end_year+1))

    columns = ['season',

               'def_explosiveness', 'def_fp_avg_pred_points', 'def_fp_avg_start',
               'def_havoc_db', 'def_havoc_7', 'def_havoc_total',
               'def_line_yards', 'def_open_field_yards',
               'def_pp_explosiveness', 'def_pp_ppa', 'def_pp_rate', 'def_pp_success_rate',
               'def_points_per_opportunity', 'def_power_success', 'def_ppa',
               'def_rp_explosiveness', 'def_rp_ppa', 'def_rp_rate', 'def_rp_success_rate',
               'def_second_level_yards', 'def_stuff_rate', 'def_success_rate', 'def_total_ppa',

               'off_explosiveness', 'off_fp_avg_pred_points', 'off_fp_avg_start',
               'off_havoc_db', 'off_havoc_7', 'off_havoc_total',
               'off_line_yards', 'off_open_field_yards',
               'off_pp_explosiveness', 'off_pp_ppa', 'off_pp_rate', 'off_pp_success_rate',
               'off_points_per_opportunity', 'off_power_success', 'off_ppa',
               'off_rp_explosiveness', 'off_rp_ppa', 'off_rp_rate', 'off_rp_success_rate',
               'off_second_level_yards', 'off_stuff_rate', 'off_success_rate', 'off_total_ppa']
    df = pd.DataFrame(columns=columns)
    for season in years:
        try:
            api_response = get_advanced_stats(season, team)

            def_explosiveness = api_response.defense.explosiveness
            def_fp_avg_pred_points = api_response.defense.field_position.average_predicted_points
            def_fp_avg_start = api_response.defense.field_position.average_start
            def_havoc_db = api_response.defense.havoc.db
            def_havoc_7 = api_response.defense.havoc.front_seven
            def_havoc_total = api_response.defense.havoc.total
            def_line_yards = api_response.defense.line_yards
            def_open_field_yards = api_response.defense.open_field_yards
            def_pp_explosiveness = api_response.defense.passing_plays.explosiveness
            def_pp_ppa = api_response.defense.passing_plays.ppa
            def_pp_rate = api_response.defense.passing_plays.rate
            def_pp_success_rate = api_response.defense.passing_plays.success_rate
            def_points_per_opportunity = api_response.defense.points_per_opportunity
            def_power_success = api_response.defense.power_success
            def_ppa = api_response.defense.ppa
            def_rp_explosiveness = api_response.defense.rushing_plays.explosiveness
            def_rp_ppa = api_response.defense.rushing_plays.ppa
            def_rp_rate = api_response.defense.rushing_plays.rate
            def_rp_success_rate = api_response.defense.rushing_plays.success_rate
            def_second_level_yards = api_response.defense.second_level_yards
            def_stuff_rate = api_response.defense.stuff_rate
            def_success_rate = api_response.defense.success_rate
            def_total_ppa = api_response.defense.total_ppa

            off_explosiveness = api_response.offense.explosiveness
            off_fp_avg_pred_points = api_response.offense.field_position.average_predicted_points
            off_fp_avg_start = api_response.offense.field_position.average_start
            off_havoc_db = api_response.offense.havoc.db
            off_havoc_7 = api_response.offense.havoc.front_seven
            off_havoc_total = api_response.offense.havoc.total
            off_line_yards = api_response.offense.line_yards
            off_open_field_yards = api_response.offense.open_field_yards
            off_pp_explosiveness = api_response.offense.passing_plays.explosiveness
            off_pp_ppa = api_response.offense.passing_plays.ppa
            off_pp_rate = api_response.offense.passing_plays.rate
            off_pp_success_rate = api_response.offense.passing_plays.success_rate
            off_points_per_opportunity = api_response.offense.points_per_opportunity
            off_power_success = api_response.offense.power_success
            off_ppa = api_response.offense.ppa
            off_rp_explosiveness = api_response.offense.rushing_plays.explosiveness
            off_rp_ppa = api_response.offense.rushing_plays.ppa
            off_rp_rate = api_response.offense.rushing_plays.rate
            off_rp_success_rate = api_response.offense.rushing_plays.success_rate
            off_second_level_yards = api_response.offense.second_level_yards
            off_stuff_rate = api_response.offense.stuff_rate
            off_success_rate = api_response.offense.success_rate
            off_total_ppa = api_response.offense.total_ppa

            df.loc[len(df)] = [season,

                                def_explosiveness, def_fp_avg_pred_points, def_fp_avg_start,
                                def_havoc_db, def_havoc_7, def_havoc_total,
                                def_line_yards, def_open_field_yards,
                                def_pp_explosiveness, def_pp_ppa, def_pp_rate, def_pp_success_rate,
                                def_points_per_opportunity, def_power_success, def_ppa,
                                def_rp_explosiveness, def_rp_ppa, def_rp_rate, def_rp_success_rate,
                                def_second_level_yards, def_stuff_rate, def_success_rate, def_total_ppa,

                                off_explosiveness, off_fp_avg_pred_points, off_fp_avg_start,
                                off_havoc_db, off_havoc_7, off_havoc_total,
                                off_line_yards, off_open_field_yards,
                                off_pp_explosiveness, off_pp_ppa, off_pp_rate, off_pp_success_rate,
                                off_points_per_opportunity, off_power_success, off_ppa,
                                off_rp_explosiveness, off_rp_ppa, off_rp_rate, off_rp_success_rate,
                                off_second_level_yards, off_stuff_rate, off_success_rate, off_total_ppa]
        except ApiException as e:
            print("Exception when calling GamesApi->get_games: %s\n" % e)
    return df

Call twice to return 2 csv's, one with team's season stats by year and the second with weekly opponents' season stats by year

In [ ]:
clemson_advanced_opp_2001_2024 = get_advanced_opp_data('Clemson', 2001, 2024)
clemson_advanced_opp_2001_2024.to_csv('clemson_advanced_opp_2001_2024.csv', index=False)

clemson_2001_2024 = get_advanced_data('Clemson', 2001, 2024)
clemson_2001_2024.to_csv('clemson_advanced_2001_2024.csv', index=False)

In [ ]:
smu_advanced_opp_2001_2024 = get_advanced_opp_data('SMU', 2001, 2024)
smu_advanced_opp_2001_2024.to_csv('smu_advanced_opp_2001_2024.csv', index=False)

smu_2001_2024 = get_advanced_data('SMU', 2001, 2024)
smu_2001_2024.to_csv('smu_advanced_2001_2024.csv', index=False)

# 3b. Get Advanced Cumulative Weekly Stats

In [ ]:
def get_advanced_weekly_stats(season, team, week):
    api_instance = cfbd.StatsApi(cfbd.ApiClient(configuration))
    try:
        if week != 1:
            api_response = api_instance.get_advanced_team_season_stats(year=season, team=team, end_week=week, exclude_garbage_time=True)
            try:
                return api_response[0]
            except IndexError:
                return None
        else:
            return None
    except ApiException as e:
        print("Exception when calling StatsApi->get_: %s\n" % e)

In [ ]:
def get_advanced_weekly_data(team, start_year, end_year):
    api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
    years = list(range(start_year, end_year+1))

    columns = ['season', 'week', 'home', 'win', f'{team.lower()}_points', 'opp', 'opp_points', 'opp_conference',

               'opp_def_explosiveness', 'opp_def_fp_avg_pred_points', 'opp_def_fp_avg_start',
               'opp_def_havoc_db', 'opp_def_havoc_7', 'opp_def_havoc_total',
               'opp_def_line_yards', 'opp_def_open_field_yards',
               'opp_def_pp_explosiveness', 'opp_def_pp_ppa', 'opp_def_pp_rate', 'opp_def_pp_success_rate',
               'opp_def_points_per_opportunity', 'opp_def_power_success', 'opp_def_ppa',
               'opp_def_rp_explosiveness', 'opp_def_rp_ppa', 'opp_def_rp_rate', 'opp_def_rp_success_rate',
               'opp_def_second_level_yards', 'opp_def_stuff_rate', 'opp_def_success_rate', 'opp_def_total_ppa',

               'opp_off_explosiveness', 'opp_off_fp_avg_pred_points', 'opp_off_fp_avg_start',
               'opp_off_havoc_db', 'opp_off_havoc_7', 'opp_off_havoc_total',
               'opp_off_line_yards', 'opp_off_open_field_yards',
               'opp_off_pp_explosiveness', 'opp_off_pp_ppa', 'opp_off_pp_rate', 'opp_off_pp_success_rate',
               'opp_off_points_per_opportunity', 'opp_off_power_success', 'opp_off_ppa',
               'opp_off_rp_explosiveness', 'opp_off_rp_ppa', 'opp_off_rp_rate', 'opp_off_rp_success_rate',
               'opp_off_second_level_yards', 'opp_off_stuff_rate', 'opp_off_success_rate', 'opp_off_total_ppa',

               'def_explosiveness', 'def_fp_avg_pred_points', 'def_fp_avg_start',
               'def_havoc_db', 'def_havoc_7', 'def_havoc_total',
               'def_line_yards', 'def_open_field_yards',
               'def_pp_explosiveness', 'def_pp_ppa', 'def_pp_rate', 'def_pp_success_rate',
               'def_points_per_opportunity', 'def_power_success', 'def_ppa',
               'def_rp_explosiveness', 'def_rp_ppa', 'def_rp_rate', 'def_rp_success_rate',
               'def_second_level_yards', 'def_stuff_rate', 'def_success_rate', 'def_total_ppa',

               'off_explosiveness', 'off_fp_avg_pred_points', 'off_fp_avg_start',
               'off_havoc_db', 'off_havoc_7', 'off_havoc_total',
               'off_line_yards', 'off_open_field_yards',
               'off_pp_explosiveness', 'off_pp_ppa', 'off_pp_rate', 'off_pp_success_rate',
               'off_points_per_opportunity', 'off_power_success', 'off_ppa',
               'off_rp_explosiveness', 'off_rp_ppa', 'off_rp_rate', 'off_rp_success_rate',
               'off_second_level_yards', 'off_stuff_rate', 'off_success_rate', 'off_total_ppa']
    df = pd.DataFrame(columns=columns)
    for year in years:
        try:
            api_response = api_instance.get_games(year=year, team=team)
            for game in api_response:
                season = game.season
                week = game.week
                if game.home_team == f"{team}":
                    home = 'True'
                    team_points = game.home_points
                    opp = game.away_team
                    opp_points = game.away_points
                else:
                    home = 'False'
                    team_points = game.away_points
                    opp = game.home_team
                    opp_points = game.home_points
                if str(team_points) != 'None' and str(opp_points) != 'None':
                    if int(str(team_points)) > int(str(opp_points)):
                        win = 'True'
                    else:
                        win = 'False'
                else:
                    win = None
                api_response = get_advanced_weekly_stats(season, opp, week)
                if api_response is not None:
                    opp_conference = api_response.conference

                    opp_def_explosiveness = api_response.defense.explosiveness
                    opp_def_fp_avg_pred_points = api_response.defense.field_position.average_predicted_points
                    opp_def_fp_avg_start = api_response.defense.field_position.average_start
                    opp_def_havoc_db = api_response.defense.havoc.db
                    opp_def_havoc_7 = api_response.defense.havoc.front_seven
                    opp_def_havoc_total = api_response.defense.havoc.total
                    opp_def_line_yards = api_response.defense.line_yards
                    opp_def_open_field_yards = api_response.defense.open_field_yards
                    opp_def_pp_explosiveness = api_response.defense.passing_plays.explosiveness
                    opp_def_pp_ppa = api_response.defense.passing_plays.ppa
                    opp_def_pp_rate = api_response.defense.passing_plays.rate
                    opp_def_pp_success_rate = api_response.defense.passing_plays.success_rate
                    opp_def_points_per_opportunity = api_response.defense.points_per_opportunity
                    opp_def_power_success = api_response.defense.power_success
                    opp_def_ppa = api_response.defense.ppa
                    opp_def_rp_explosiveness = api_response.defense.rushing_plays.explosiveness
                    opp_def_rp_ppa = api_response.defense.rushing_plays.ppa
                    opp_def_rp_rate = api_response.defense.rushing_plays.rate
                    opp_def_rp_success_rate = api_response.defense.rushing_plays.success_rate
                    opp_def_second_level_yards = api_response.defense.second_level_yards
                    opp_def_stuff_rate = api_response.defense.stuff_rate
                    opp_def_success_rate = api_response.defense.success_rate
                    opp_def_total_ppa = api_response.defense.total_ppa

                    opp_off_explosiveness = api_response.offense.explosiveness
                    opp_off_fp_avg_pred_points = api_response.offense.field_position.average_predicted_points
                    opp_off_fp_avg_start = api_response.offense.field_position.average_start
                    opp_off_havoc_db = api_response.offense.havoc.db
                    opp_off_havoc_7 = api_response.offense.havoc.front_seven
                    opp_off_havoc_total = api_response.offense.havoc.total
                    opp_off_line_yards = api_response.offense.line_yards
                    opp_off_open_field_yards = api_response.offense.open_field_yards
                    opp_off_pp_explosiveness = api_response.offense.passing_plays.explosiveness
                    opp_off_pp_ppa = api_response.offense.passing_plays.ppa
                    opp_off_pp_rate = api_response.offense.passing_plays.rate
                    opp_off_pp_success_rate = api_response.offense.passing_plays.success_rate
                    opp_off_points_per_opportunity = api_response.offense.points_per_opportunity
                    opp_off_power_success = api_response.offense.power_success
                    opp_off_ppa = api_response.offense.ppa
                    opp_off_rp_explosiveness = api_response.offense.rushing_plays.explosiveness
                    opp_off_rp_ppa = api_response.offense.rushing_plays.ppa
                    opp_off_rp_rate = api_response.offense.rushing_plays.rate
                    opp_off_rp_success_rate = api_response.offense.rushing_plays.success_rate
                    opp_off_second_level_yards = api_response.offense.second_level_yards
                    opp_off_stuff_rate = api_response.offense.stuff_rate
                    opp_off_success_rate = api_response.offense.success_rate
                    opp_off_total_ppa = api_response.offense.total_ppa
                    try:
                        api_response = get_advanced_weekly_stats(season, team, week)
                        if api_response is None:
                            continue

                        def_explosiveness = api_response.defense.explosiveness
                        def_fp_avg_pred_points = api_response.defense.field_position.average_predicted_points
                        def_fp_avg_start = api_response.defense.field_position.average_start
                        def_havoc_db = api_response.defense.havoc.db
                        def_havoc_7 = api_response.defense.havoc.front_seven
                        def_havoc_total = api_response.defense.havoc.total
                        def_line_yards = api_response.defense.line_yards
                        def_open_field_yards = api_response.defense.open_field_yards
                        def_pp_explosiveness = api_response.defense.passing_plays.explosiveness
                        def_pp_ppa = api_response.defense.passing_plays.ppa
                        def_pp_rate = api_response.defense.passing_plays.rate
                        def_pp_success_rate = api_response.defense.passing_plays.success_rate
                        def_points_per_opportunity = api_response.defense.points_per_opportunity
                        def_power_success = api_response.defense.power_success
                        def_ppa = api_response.defense.ppa
                        def_rp_explosiveness = api_response.defense.rushing_plays.explosiveness
                        def_rp_ppa = api_response.defense.rushing_plays.ppa
                        def_rp_rate = api_response.defense.rushing_plays.rate
                        def_rp_success_rate = api_response.defense.rushing_plays.success_rate
                        def_second_level_yards = api_response.defense.second_level_yards
                        def_stuff_rate = api_response.defense.stuff_rate
                        def_success_rate = api_response.defense.success_rate
                        def_total_ppa = api_response.defense.total_ppa

                        off_explosiveness = api_response.offense.explosiveness
                        off_fp_avg_pred_points = api_response.offense.field_position.average_predicted_points
                        off_fp_avg_start = api_response.offense.field_position.average_start
                        off_havoc_db = api_response.offense.havoc.db
                        off_havoc_7 = api_response.offense.havoc.front_seven
                        off_havoc_total = api_response.offense.havoc.total
                        off_line_yards = api_response.offense.line_yards
                        off_open_field_yards = api_response.offense.open_field_yards
                        off_pp_explosiveness = api_response.offense.passing_plays.explosiveness
                        off_pp_ppa = api_response.offense.passing_plays.ppa
                        off_pp_rate = api_response.offense.passing_plays.rate
                        off_pp_success_rate = api_response.offense.passing_plays.success_rate
                        off_points_per_opportunity = api_response.offense.points_per_opportunity
                        off_power_success = api_response.offense.power_success
                        off_ppa = api_response.offense.ppa
                        off_rp_explosiveness = api_response.offense.rushing_plays.explosiveness
                        off_rp_ppa = api_response.offense.rushing_plays.ppa
                        off_rp_rate = api_response.offense.rushing_plays.rate
                        off_rp_success_rate = api_response.offense.rushing_plays.success_rate
                        off_second_level_yards = api_response.offense.second_level_yards
                        off_stuff_rate = api_response.offense.stuff_rate
                        off_success_rate = api_response.offense.success_rate
                        off_total_ppa = api_response.offense.total_ppa

                        df.loc[len(df)] = [season, week, home, win, team_points, opp, opp_points, opp_conference,

                                            opp_def_explosiveness, opp_def_fp_avg_pred_points, opp_def_fp_avg_start,
                                            opp_def_havoc_db, opp_def_havoc_7, opp_def_havoc_total,
                                            opp_def_line_yards, opp_def_open_field_yards,
                                            opp_def_pp_explosiveness, opp_def_pp_ppa, opp_def_pp_rate, opp_def_pp_success_rate,
                                            opp_def_points_per_opportunity, opp_def_power_success, opp_def_ppa,
                                            opp_def_rp_explosiveness, opp_def_rp_ppa, opp_def_rp_rate, opp_def_rp_success_rate,
                                            opp_def_second_level_yards, opp_def_stuff_rate, opp_def_success_rate, opp_def_total_ppa,

                                            opp_off_explosiveness, opp_off_fp_avg_pred_points, opp_off_fp_avg_start,
                                            opp_off_havoc_db, opp_off_havoc_7, opp_off_havoc_total,
                                            opp_off_line_yards, opp_off_open_field_yards,
                                            opp_off_pp_explosiveness, opp_off_pp_ppa, opp_off_pp_rate, opp_off_pp_success_rate,
                                            opp_off_points_per_opportunity, opp_off_power_success, opp_off_ppa,
                                            opp_off_rp_explosiveness, opp_off_rp_ppa, opp_off_rp_rate, opp_off_rp_success_rate,
                                            opp_off_second_level_yards, opp_off_stuff_rate, opp_off_success_rate, opp_off_total_ppa,

                                            def_explosiveness, def_fp_avg_pred_points, def_fp_avg_start,
                                            def_havoc_db, def_havoc_7, def_havoc_total,
                                            def_line_yards, def_open_field_yards,
                                            def_pp_explosiveness, def_pp_ppa, def_pp_rate, def_pp_success_rate,
                                            def_points_per_opportunity, def_power_success, def_ppa,
                                            def_rp_explosiveness, def_rp_ppa, def_rp_rate, def_rp_success_rate,
                                            def_second_level_yards, def_stuff_rate, def_success_rate, def_total_ppa,

                                            off_explosiveness, off_fp_avg_pred_points, off_fp_avg_start,
                                            off_havoc_db, off_havoc_7, off_havoc_total,
                                            off_line_yards, off_open_field_yards,
                                            off_pp_explosiveness, off_pp_ppa, off_pp_rate, off_pp_success_rate,
                                            off_points_per_opportunity, off_power_success, off_ppa,
                                            off_rp_explosiveness, off_rp_ppa, off_rp_rate, off_rp_success_rate,
                                            off_second_level_yards, off_stuff_rate, off_success_rate, off_total_ppa]
                    except ApiException as e:
                        print("Exception when calling GamesApi->get_games: %s\n" % e)
        except ApiException as e:
            print("Exception when calling GamesApi->get_games: %s\n" % e)
    return df

Returns 1 csv file with cumulative season stats for both opponent and team.

In [ ]:
smu_weekly_2001_2024 = get_advanced_weekly_data('SMU', 2001, 2024)
smu_weekly_2001_2024.to_csv('smu_advanced_weekly_2001_2024.csv', index=False)

In [ ]:
clemson_weekly_2001_2024 = get_advanced_weekly_data('Clemson', 2001, 2024)
clemson_weekly_2001_2024.to_csv('clemson_advanced_weekly_2001_2024.csv', index=False)